In [1]:
import os
import random
import operator
import requests
import numpy as np
import pandas as pd
from scipy import sparse
import sys
from surprise import Dataset, Reader
from surprise import KNNBasic, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy
from surprise.dataset import DatasetAutoFolds

In [2]:
df1 = pd.read_csv('Book reviews\BX-Users.csv', sep=';', encoding='ISO-8859-1')
df2 = pd.read_csv('Book reviews\BX_Books.csv', sep=';', encoding='ISO-8859-1')
df3 = pd.read_csv('Book reviews\BX-Book-Ratings.csv', sep=';', encoding='ISO-8859-1', nrows=20_000)

In [3]:
user_ids = df3['User-ID'].tolist()
user_id = []
for i in user_ids:
    if i in user_id:
        continue
    else:
        user_id.append(i)
len(user_id)

2180

In [4]:
data = df3

In [5]:
df3['Book-Rating'].value_counts()

0     12660
8      1694
7      1526
10     1272
9      1105
5       728
6       663
4       170
3       108
2        45
1        29
Name: Book-Rating, dtype: int64

In [6]:
n=len(df3)
N=list(range(n))
random.seed(2023)
random.shuffle(N)

In [7]:
train=data.iloc[N[0:(n*4)//5]]
test=data.iloc[N[(n*4)//5:]]
train.tail()

,User-ID,ISBN,Book-Rating
15849,2442,8845252906,0
11349,712,3784419445,8
1732,277427,0553579274,0
18333,3363,0553213164,10
11806,882,0553801945,0


In [8]:
print(train['Book-Rating'].unique().tolist())
print(len(train['User-ID'].unique().tolist()))
print(len(train['ISBN'].unique().tolist()))

[0, 9, 10, 2, 7, 5, 8, 6, 1, 4, 3]
1912
14033


In [9]:
reader = Reader(rating_scale=(1,10)) # rating scale range
trainset = Dataset.load_from_df(train[['User-ID','ISBN','Book-Rating']],reader).build_full_trainset()
print(type(trainset))

<class 'surprise.trainset.Trainset'>


In [10]:
# Use the KNNBasic algorithm to train the model
algo = KNNBasic()
#algo = SVD()
algo.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [11]:
testset = Dataset.load_from_df(test[['User-ID','ISBN','Book-Rating']],reader).build_full_trainset().build_anti_testset()

In [12]:
items = trainset.build_anti_testset()

In [13]:
books = df2
mapping_dict = books.set_index("ISBN")["Book-Title"].to_dict()

In [14]:
users=test['User-ID'].tolist()

In [29]:
random.seed()
rd = random.randint(0,len(users))
users[rd]

1928

In [30]:
user = users[rd]
user_items = list(filter(lambda x: x[0] == user, items))
recommendations = algo.test(user_items)
recommendations.sort(key=operator.itemgetter(3), reverse=True)

In [31]:
print(f"User {user} Recommendation Top 5:")
for r in recommendations[0:5]:
    try:  
        print(f" [Item] {mapping_dict[r[1]]}, [Estimated Rating] {round(r[3],3)}")
    except:
        continue

User 1928 Recommendation Top 5:
 [Item] Four Blind Mice, [Estimated Rating] 10
 [Item] KJV Giant Print Reference Bible, Personal Size Bronze Edition, [Estimated Rating] 10
 [Item] So You Want to Be a Wizard: The First Book in the Young Wizards Series, [Estimated Rating] 10
 [Item] The Princess Diaries, [Estimated Rating] 10
 [Item] Memoirs of a Geisha, [Estimated Rating] 10
